In [1]:
import sys
sys.path.append('./')

In [ ]:
import torch
import json
import pandas as pd
import matplotlib.pyplot as plt
from src.utils_ import set_seed, evaluate_aerofoil_predictions, find_file, read_coords
from src.models_ import instantiate_model
from src.data_ import DataPrep
from src.db import DataDB

from numpy import set_printoptions
set_printoptions(precision=4, suppress=True)

set_seed(42)

In [3]:
def filter_names(prefix: str = '', set: str = 'test') -> list:
    names_path = find_file('names.json', 'lap')
    names = json.load(open(names_path, "r"))
    if set == 'train':
        return [name for name in sorted(names['train_names']) if name.startswith(prefix)]
    if set == 'test':
        return [name for name in sorted(names['test_names']) if name.startswith(prefix)]

In [4]:
CSV_PATH = './data/csv/14KP_48CLA.csv'

data_prep = DataPrep(csv_path=CSV_PATH)
data = data_prep.get_data()

In [5]:
db = DataDB()
params = db.query_single(collection=DataDB.TUNE_COLL, network='GRUModel')

In [ ]:
model = instantiate_model(net_name=params['network'], input_size=data['input_size'], output_size=data['output_size'], params=params['best_params'])
model.load_state_dict(torch.load('./models/log_cosh/GRUModel.pt', map_location=torch.device('cpu')))

In [7]:
clark = filter_names("clar", "test")

In [ ]:
clark

In [ ]:
import re

eppler = []
for af in e:
    if re.match(r'^e\d+.*', af, re.IGNORECASE):
        eppler.append(af)

In [ ]:
len(eppler)

In [ ]:
def plot_curve(aeros: list) -> dict:
    df = pd.read_csv(CSV_PATH)

    curve_dict = {}

    for aero in aeros:
        temp = df[df['aerofoil_name'] == aero]

        alpha_values = [f"alpha_{i}" for i in range(48)]
        cl_values = [f"CL_{i}" for i in range(48)]
        x = temp[alpha_values].values.flatten().tolist()
        y = temp[cl_values].values.flatten().tolist()

        plt.figure(figsize=(8, 6))
        plt.plot(x, y, '-', label=aero)
        plt.xlabel("Angle of Attack (Alpha)")
        plt.ylabel("Lift Coefficient (CL)")
        plt.legend()
        plt.grid(True)
        plt.show()

        curve_dict[aero] = {'alpha': x, 'CL': y}

    return curve_dict


def plot_aero(aeros):
    for aero in aeros:
        coords = read_coords(aero)
        
        x = [x[0] for x in coords]
        y = [x[1] for x in coords]

        plt.style.use('bmh')

        plt.ylim((-0.5, 0.5))
        plt.plot(x, y, label=f'{aero.upper()}')
        plt.legend(fontsize='small')
        plt.show()


In [ ]:
aeros = ['naca64a210', 'drgnfly', 'fx67k150', 'naca0024', 'naca65206', 'ag19', 'ah93w300']
plot_aero(aeros)

In [ ]:
cd = plot_curve(aeros)

In [ ]:
res = evaluate_aerofoil_predictions(
    csv_file=CSV_PATH,
    model=model,
    X_scaler=data['X_scaler'],
    y_scaler=data['y_scaler'],
    plot_sample=['naca2410']
)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('../../data/csv/14KP_48CLA.csv')

In [ ]:
naca2410_df = df[df['aerofoil_name'] == 'naca2410'][[f'alpha_{i}' for i in range(48)] + [f'CL_{i}' for i in range(48)]]

In [ ]:
predictions = res['naca2410']['predictions']
for i in range(48):
    print(f'Alpha {i}:', round(predictions[i+48], 4), f'CL {i}:', round(predictions[i], 4))


In [ ]:
preds = {}
for i in range(len(predictions)//2):
    preds[i] = {
        'alpha': round(predictions[i+48], 4),
        'CL': round(predictions[i], 4)
    }

In [ ]:
for i in preds:
    print(f'Alpha {i}:', preds[i]['alpha'], f'CL {i}:', preds[i]['CL'])

In [13]:
import numpy as np

u_inf = 14.71
alphas = [i for i in range(-15, 19)]
alphas = np.array(alphas)

In [14]:
components = {}
for alpha in alphas:
    components[int(alpha)] = {
        'drag_components': {
            'x': float(round(np.cos(np.deg2rad(alpha)), 6)),
            'y': float(round(np.sin(np.deg2rad(alpha)), 6))
        },
        'lift_components': {
            'x': float(round(-np.sin(np.deg2rad(alpha)), 6)),
            'y': float(round(np.cos(np.deg2rad(alpha)), 6))
        },
        'vel_components': {
            'x': float(round(u_inf * np.cos(np.deg2rad(alpha)), 6)),
            'y': float(round(u_inf * np.sin(np.deg2rad(alpha)), 6)),
        }
    }

In [15]:
def get_components(alpha):
    print(f"Alpha: {alpha}")
    
    print(f"Drag Components: X: {components[alpha]['drag_components']['x']}, Y: {components[alpha]['drag_components']['y']}")
    print(f"Lift Components: X: {components[alpha]['lift_components']['x']}, Y: {components[alpha]['lift_components']['y']}")
    print(f"Velocity Components: X: {components[alpha]['vel_components']['x']}, Y: {components[alpha]['vel_components']['y']}")

In [ ]:
get_components(-9)

In [16]:
cfd = {
    'u_inf': 14.71,
    'components': components,
    'results': {
        -8: -0.63221,
        -7: -0.53020,
        -6: -0.42648,
        -5: -0.32146,
        -4: -0.21557,
        -3: -0.10915,
        -2: -0.0024834,
        -1: 0.10419,
        0: 0.21037,
        1: 0.31696,
        2: 0.42302,
        3: 0.52855,
        4: 0.63308,
        5: 0.73606,
        6: 0.83716,
        7: 0.93613,        
        8: 1.0323,
        9: 1.1247,
        10: 1.2124,
        11: 1.2942,
        12: 1.3682,
        13: 1.4318,
        14: 1.4814,
        15: 1.5113,
        16: 1.2439,
        17: 1.6863,
    }
}

t = 10.36
F = 161.16

In [18]:
import matplotlib.pyplot as plt
x = predictions[48:]
y = predictions[:48]
plt.plot([i for i in range(-8, 18)], [cfd['results'][i] for i in range(-8, 18)])
plt.plot(x, y, '--')
plt.show()

In [ ]:
for i in range(48):
    if (preds[i]['alpha'] // 1) == float(i // 1) - 15:
        print(f'Alpha {i}: {preds[i]["CL"]}', f'CFD: {cfd["results"][i]}')

In [ ]:
cfd["results"][0]